# **combatlearn**: Quick‑Start Demo
This notebook walks through the main **combatlearn** features:
ComBat (Johnson _et al._ 2007), neuroComBat (Fortin _et al._ 2018), and CovBat (Chen _et al._ 2022).

```bash
# If you haven't installed the package yet, run this:
pip install combatlearn
```

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from combatlearn import ComBat

plt.rcParams['figure.dpi'] = 120


In [ ]:
def plot_pca(X, labels, title):
    pca = PCA(n_components=2)
    pts = pca.fit_transform(X)
    fig = plt.figure(dpi=150)
    for b in np.unique(labels):
        idx = labels == b
        plt.scatter(pts[idx, 0], pts[idx, 1], label=f"batch {b}")
    plt.title(title)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.legend()
    plt.show()


## Original data

In [ ]:
# Reproducibility
rng = np.random.default_rng(42)

n_samples, n_features, n_batches = 300, 50, 3
true_signal = rng.standard_normal((n_samples, n_features))

# Inject batch‑specific mean shifts
batch_effects = rng.normal(
    loc=np.array([0, 3, -2])[:, None],  # shape (3,1) for broadcasting
    scale=0.5,
    size=(n_batches, n_features)
)

batches = np.repeat(np.arange(n_batches), n_samples // n_batches)
X = true_signal.copy()

for b in range(n_batches):
    X[batches == b] += batch_effects[b]

X = pd.DataFrame(X, columns=[f"gene_{i+1}" for i in range(n_features)])
batch_labels = pd.Series(batches.astype(str), name='batch')

plot_pca(X.values, batch_labels.values, title='Before correction')

In [ ]:
# Simulate a categorical (sex) and continuous (age) covariate
sex = pd.Series(rng.choice(['M', 'F'], size=n_samples), name='sex')
age = pd.Series(rng.uniform(20, 60, size=n_samples), name='age')

## The three methods

### Johnson method

In [ ]:
combat_johnson = ComBat(batch=batch_labels, method='johnson')
X_johnson = combat_johnson.fit_transform(X)

plot_pca(X_johnson.values, batch_labels.values, title='After Johnson')

Johnson's method ignores the covariates

In [ ]:
combat_johnson_wc = ComBat(
    batch=batch_labels,
    discrete_covariates=sex,
    continuous_covariates=age,
    method='johnson',
)
X_johnson_wc = combat_johnson_wc.fit_transform(X)

In [ ]:
print("Are all values equal?", "Yes" if np.allclose(X_johnson, X_johnson_wc) else "No")

### Fortin method

In [ ]:
combat_fortin = ComBat(
    batch=batch_labels,
    discrete_covariates=sex,
    continuous_covariates=age,
    method='fortin'
)
X_fortin = combat_fortin.fit_transform(X)

plot_pca(X_fortin.values, batch_labels.values, title='After Fortin')

### Chen method

In [ ]:
combat_chen = ComBat(
    batch=batch_labels,
    discrete_covariates=sex,
    continuous_covariates=age,
    method='chen',
    covbat_cov_thresh=0.95
)
X_chen = combat_chen.fit_transform(X)

plot_pca(X_chen.values, batch_labels.values, title='After Chen')

## Parametric versus non parametric approach

In [ ]:
combat_parametric = ComBat(
    batch=batch_labels,
    method='johnson',
    parametric=True
)
X_parametric = combat_parametric.fit_transform(X)
plot_pca(X_parametric.values, batch_labels.values, title='After Parametric Johnson')
combat_non_parametric = ComBat(
    batch=batch_labels,
    method='johnson',
    parametric=False
)
X_non_parametric = combat_non_parametric.fit_transform(X)
plot_pca(X_non_parametric.values, batch_labels.values, title='After Non-parametric Johnson')

## Working with `sklearn`

Creating target

In [ ]:
# Example: classification with a batch‑aware preprocessing pipeline
# (Here we create a toy binary label correlated with the true signal.)
y = (true_signal[:, 0] > 0).astype(int)

### Effect of ComBat in training set and test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

combat = ComBat(batch=batch_labels, method='johnson')
combat.fit(X_train)

plot_pca(X_train.values, batch_labels.iloc[X_train.index].values, title='Train set before ComBat')

X_train_combat = combat.transform(X_train)
plot_pca(X_train_combat.values, batch_labels.iloc[X_train_combat.index].values, title='Train set after ComBat')

In [ ]:
plot_pca(X_test.values, batch_labels.iloc[X_test.index].values, title='Test set before ComBat')

X_test_combat = combat.transform(X_test)
plot_pca(X_test_combat.values, batch_labels.iloc[X_test_combat.index].values, title='Test set after ComBat')

### Use of ComBat within a scikit-learn `Pipeline`

In [ ]:
pipe = Pipeline([
    ('combat', ComBat(batch=batch_labels, method='johnson')),
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression())
])

scores = cross_val_score(pipe, X, y, cv=5)
print(f'Cross‑validated accuracy: {scores.mean():.3f} ± {scores.std():.3f}')